In [107]:
import numpy as np
import pandas as pd
import operator
import re
import time
import random
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import HiveContext

In [108]:
if 'sc' not in locals().keys():
    sc = SparkContext(appName="xxx")
sqlContext = HiveContext(sc)

In [109]:
# def func_get_feat_vec(line):
#     line = line.strip().split('\t')
#     return [1 if feat+':1' in line[2:] else 0 for feat in bc_fname.value]

# def func_is_nega(line):
#     line = line.strip().split('\t')
#     return int(line[1])==0

def func_get_featlabel_vec(line):
    line = line.strip().split('\t')
    return ','.join([str(line[1])] + ['1' if feat+':1' in line[2:] else '0' for feat in bc_fname.value])

In [110]:
curdate = '20170803'
for cid in ['200000360']:
    str_tr_path = '../data/baseline/'+curdate+'/tr_'+str(cid)
    str_te_path = '../data/baseline/'+curdate+'/te_'+str(cid)
    str_fe_path = '../data/baseline/'+curdate+'/feat_weight_'+str(cid)
    
    print str_fe_path
    sorted_fname = sorted(sc.textFile(str_fe_path).map(lambda line:str(line.split('\t')[1])).collect())
    sorted_fname.remove('CVR')
    sorted_fname.remove('F0')
    sorted_fname.remove('ratio')
    sorted_fname = sorted_fname[0:10]
    print sorted_fname
    bc_fname = sc.broadcast(sorted_fname)

    rdd_train = sc.textFile(str_tr_path,minPartitions=500).sample(False,0.05,823).map(func_get_featlabel_vec)
    #sparkdf_train = sqlContext.createDataFrame(rdd_train, schema=bc_fname.value+['LABEL'])
    #pddf_train = sparkdf_train.toPandas()

../data/baseline/20170803/feat_weight_200000360
['Server', 'cvt', 'cvt_10414', 'cvtday0', 'cvtday1', 'cvtday10', 'cvtday11', 'cvtday12', 'cvtday13', 'cvtday14']


In [117]:
def readPointBatch(iterator):
    samples = list(iterator)
    matrix = np.zeros((len(samples),1 + np.shape(sorted_fname)[0]))
    for index, value in enumerate(samples):
        matrix[index] = np.fromstring(value.replace(',',' '), dtype=np.float128, sep=' ')
    return [matrix]

def gradient(matrix, weights):
    Y = matrix[:, 0]
    X = matrix[:, 1:]
    A = np.ones(X.shape[0]) # Intercept
    X = np.c_[A, X]
    
    #return ((1.0 / (1.0 + np.exp(-Y * X.dot(weights))) - 1.0) * Y * X.T).sum(1)
    return ((1.0 / (1.0 + np.exp(-X.dot(weights))) - Y) * X.T).sum(1) / X.shape[0]

def cost(matrix, weights):
    Y = matrix[:, 0]
    X = matrix[:, 1:]
    A = np.ones(X.shape[0]) # Intercept
    X = np.c_[A, X]
    
    h = lambda X: 1.0 / (1 + np.exp(-X.dot(weights)))
    return np.average(-(Y * np.log(h(X)) + (1.0 - Y) * np.log(1.0 - h(X))))

def add(x,y):
    x += y
    return x

In [112]:
points = rdd_train.mapPartitions(readPointBatch).cache()
print np.shape(sorted_fname)
print np.shape(points.first())
print rdd_train.count()

(10,)
(75, 11)
46934


In [118]:
maxk = 5000
rho = 0.5
sigma = 0.4
epsilon = 1e-5
k = 0

D = np.shape(sorted_fname)[0]
weights = 2 * np.random.ranf(size=D+1) - 1
# print("Initial w: " + str(weights))
while k < maxk:
    
    gk = points.map(lambda matrix: gradient(matrix, weights)).reduce(add) / 500
    dk = -gk
    
    if k % 50 == 0:
        print("On iteration %i" % (k + 1))
        print weights
        print dk
        print 'norm(dk):\t' + str(np.linalg.norm(dk))
        print
        
    if np.linalg.norm(dk) < epsilon:
        break
        
    if k > 100:
        m = 0
        mk = 0
        while m < 20:
            current_cost1 = points.map(lambda matrix: cost(matrix, weights)).reduce(add) / 500
            current_cost2 = points.map(lambda matrix: cost(matrix, weights + rho**m * dk)).reduce(add) / 500
            print 'current_cost1:\t' + str(current_cost1)
            print 'current_cost2:\t' + str(current_cost2)
            print sigma * rho**m * np.dot(gk,dk)
            if current_cost2 <= current_cost1 + sigma * rho**m * np.dot(gk,dk):
                mk = m
                break
            m += 1

        print 'mk:\t' + str(mk)
        print 'alpha:\t' + str(rho**mk)
        weights += rho**mk * dk
    else:
        weights += dk
        
    k += 1

On iteration 1
[-0.51780155 -0.95801979  0.07476617 -0.31162802 -0.16465073 -0.99727857
 -0.40465144  0.54435415 -0.68102634 -0.33778926  0.24790515]
[ -3.72940353e-01  -0.00000000e+00  -2.58004853e-04  -2.58004853e-04
  -6.26588276e-06   1.20294940e-05  -2.54009411e-05   5.12232845e-06
  -1.03430271e-05  -2.09006287e-05  -3.35686001e-05]
norm(dk):	0.372940534955

On iteration 51
[-3.89900499 -0.95801979  0.07773143 -0.30866275 -0.16470331 -0.9962335
 -0.40486116  0.54528426 -0.68110675 -0.33796161  0.24848429]
[ -1.95157399e-02  -0.00000000e+00   1.06688733e-04   1.06688733e-04
  -2.95778047e-07   2.20547715e-05  -1.17065543e-06   2.09299510e-05
  -4.34443482e-07  -9.60852445e-07   1.88702999e-05]
norm(dk):	0.0195163559744

On iteration 101
[-4.58224085 -0.95801979  0.08334333 -0.30305086 -0.16471377 -0.99512464
 -0.40490256  0.54634589 -0.6811221  -0.33799558  0.24947292]
[ -9.78746696e-03  -0.00000000e+00   1.15908398e-04   1.15908398e-04
  -1.52047638e-07   2.22580322e-05  -6.01647

current_cost1:	0.00785205627338
current_cost2:	0.00781913536384
-1.32081676441e-05
mk:	0
alpha:	1.0
current_cost1:	0.00781913536384
current_cost2:	0.007786609167
-1.30495821277e-05
mk:	0
alpha:	1.0
current_cost1:	0.007786609167
current_cost2:	0.00775447075562
-1.28937837491e-05
mk:	0
alpha:	1.0
current_cost1:	0.00775447075562
current_cost2:	0.00772271336275
-1.27407079379e-05
mk:	0
alpha:	1.0
current_cost1:	0.00772271336275
current_cost2:	0.00769133037707
-1.25902919781e-05
mk:	0
alpha:	1.0
current_cost1:	0.00769133037707
current_cost2:	0.00766031533851
-1.24424749449e-05
mk:	0
alpha:	1.0
current_cost1:	0.00766031533851
current_cost2:	0.00762966193391
-1.22971976434e-05
mk:	0
alpha:	1.0
current_cost1:	0.00762966193391
current_cost2:	0.00759936399293
-1.21544025505e-05
mk:	0
alpha:	1.0
current_cost1:	0.00759936399293
current_cost2:	0.00756941548398
-1.20140337582e-05
mk:	0
alpha:	1.0
current_cost1:	0.00756941548398
current_cost2:	0.00753981051041
-1.18760369198e-05
mk:	0
alpha:	1.0
curr

current_cost1:	0.00608765955445
current_cost2:	0.00607271260969
-5.99127515156e-06
mk:	0
alpha:	1.0
current_cost1:	0.00607271260969
current_cost2:	0.00605788991776
-5.94142245021e-06
mk:	0
alpha:	1.0
On iteration 251
[-5.47495931 -0.95801979  0.10127215 -0.28512204 -0.16472814 -0.99177406
 -0.40495941  0.54959092 -0.68114312 -0.33804224  0.25261726]
[ -3.83398943e-03  -0.00000000e+00   1.21516866e-04   1.21516866e-04
  -6.48001640e-08   2.23806572e-05  -2.56538750e-07   2.17436937e-05
  -9.47744816e-08  -2.10532929e-07   2.12883266e-05]
norm(dk):	0.00383802483254

current_cost1:	0.00605788991776
current_cost2:	0.00604318997448
-5.89217384608e-06
mk:	0
alpha:	1.0
current_cost1:	0.00604318997448
current_cost2:	0.00602861129965
-5.84351969291e-06
mk:	0
alpha:	1.0
current_cost1:	0.00602861129965
current_cost2:	0.0060141524366
-5.79545053559e-06
mk:	0
alpha:	1.0
current_cost1:	0.0060141524366
current_cost2:	0.00599981195171
-5.74795710564e-06
mk:	0
alpha:	1.0
current_cost1:	0.00599981195171

KeyboardInterrupt: 

In [85]:
print("Final w: " + str(weights))

Final w: [ -2.41989354e+04  -4.42228394e-03   1.18577690e+02   1.19713920e+02
   4.57270834e-01  -7.43367457e-01  -7.27528791e+00  -6.56101965e-01
  -6.97503798e-01  -3.66567336e-01   9.93411857e-01]
